In [1]:
import os
import absl
import urllib
import tempfile

import tensorflow as tf
import tfx

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

tf.get_logger().propogate = False
absl.logging.set_verbosity(absl.logging.INFO)

In [6]:
_data_root = tempfile.mkdtemp()
_data_filepath = os.path.join(_data_root, 'data.csv')
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

('/tmp/tmpjklcixjq/data.csv', <http.client.HTTPMessage at 0x7f099a5e14d0>)

In [7]:
context = InteractiveContext()

## How to use an ExampleGen Component

In [8]:
from tfx.utils.dsl_utils import external_input
from tfx.components import CsvExampleGen

examples = external_input(_data_root)
example_gen = CsvExampleGen(
    input=examples,
#     input_config=,
#     output_config=,
#     example_artifacts=,
#     instance_name=,
#     enable_cache=,
)
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Processing input csv data /tmp/tmpjklcixjq/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-05-14T20_33_45.785358-9hn8a_ez/CsvExampleGen/examples/1, id: 2)]
        ))

## Span, Version and Split

### Custom input/output split

In [11]:
from tfx.proto import example_gen_pb2

output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(
        splits=[
            example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=70),
            example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=20),
            example_gen_pb2.SplitConfig.Split(name='text', hash_buckets=10)
        ]
    )
)
example_gen = CsvExampleGen(
    input=examples,
    output_config=output_config
)
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Processing input csv data /tmp/tmpjklcixjq/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 3
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-05-14T20_33_45.785358-9hn8a_ez/CsvExampleGen/examples/3, id: 4)]
        ))

In [ ]:
# For an input source which has already been split, set the input_config for ExampleGen component.
# Input train split is 'input_dir/train/*', eval split is 'input_dir/eval/*'.
# Output splits are generated one-to-one mapping from input splits.
input_config = example_gen_pb2.Input(
    splits=[
        example_gen_pb2.Input.Split(name='train', pattern='train/*'),
        example_gen_pb2.Input.Split(name='eval', pattern='eval/*')
    ]
)

example_gen = CsvExampleGen(
    input=examples,
    input_config=input_config
)
context.run(example_gen)

### Span

Span can be retrieved by using '{SPAN}' spec in the input glob pattern:

- This spec matches digits and maps the data into the relevant SPAN numbers. For example, 'data_{SPAN}-*.tfrecord' will collect files like 'data_12-a.tfrecord', 'date_12-b.tfrecord'.
- When SPAN spec is missing, it's assumed to be always Span '0'.
- If SPAN is specified, pipeline will process the latest span, and store the span number in metadata

For example, let's assume there are input data:

- '/tmp/span-01/train/data'
- '/tmp/span-01/eval/data'
- '/tmp/span-02/train/data'
- '/tmp/span-02/eval/data'

and the input config is shown as below:

```python
input_config = example_gen_pb2.Input(
    splits=[
        example_gen_pb2.Input.Split(name='train', pattern='span-{SPAN}/train/*'),
        example_gen_pb2.Input.Split(name='eval', pattern='span-{SPAN}/eval/*'),
    ]
)
```

Then it will process:

- '/tmp/span-02/train/data' as train split
- '/tmp/span-02/eval/data' as eval split

## Custom ExampleGen

### File-Based ExampleGen

First, extend BaseExampleGenExecutor with a custom Beam PTransform, which provides the conversion from your train/eval input split to TF examples. For example, the CsvExampleGen executor provides the conversion from an input CSV split to TF examples.

Then, create a component with above executor, as done in CsvExampleGen component. Alternatively, pass a custom executor into the standard ExampleGen component as shown below.

In [15]:
from tfx.components.base import executor_spec
from tfx.components import FileBasedExampleGen
from tfx.components.example_gen.csv_example_gen import executor
example_gen = FileBasedExampleGen(
    input=examples,
    custom_executor_spec=executor_spec.ExecutorClassSpec(executor.Executor)
)
context.run(example_gen)

INFO:absl:Running driver for FileBasedExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for FileBasedExampleGen
INFO:absl:Generating examples.
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Processing input csv data /tmp/tmpjklcixjq/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for FileBasedExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: FileBasedExampleGen
    execution_id: 4
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-05-14T20_33_45.785358-9hn8a_ez/FileBasedExampleGen/examples/4, id: 5)]
        ))

### Query-Based ExampleGen